<a href="https://colab.research.google.com/github/pravin691983/insaid-capstone_project_1/blob/development/SourceCode/insaid_capstone_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSAID - Capstone Project on Data Analytics**

In [1]:
print("Welcome to INSAID - Capstone Project on Data Analytics")

Welcome to INSAID - Capstone Project on Data Analytics


# **Importing a library that is not in Colaboratory**

To import a library that's not in Colaboratory by default, you can use !pip install or !apt-get install.

In [2]:
!pip install -q mysql-connector-python-rf

     |████████████████████████████████| 11.9MB 340kB/s 


In [0]:
#Import Required packages
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [5]:

#Connect to MySQL
connection = mysql.connector.connect(host='cpanel.insaid.co',
                                     database='Capstone1',
                                         user='student',
                                         password='student')
if connection.is_connected():
    db_Info = connection.get_server_info()
    print("Connected to MySQL Server version ", db_Info)
    cursor = connection.cursor()
    cursor.execute("select database();")
    record = cursor.fetchone()
    print("You're connected to database: ", record)

Connected to MySQL Server version  5.6.46-cll-lve
You're connected to database:  ('Capstone1',)
